In [22]:
import requests
import pandas as pd
import pgeocode
import ee
import time
from datetime import datetime
from tqdm import tqdm

ee.Authenticate()
ee.Initialize(project='machinelearningcommodity')

In [23]:
longlat = pd.read_csv("zip_df.csv")
longlat = longlat.iloc[:, :-1]
longlat = longlat.dropna()
longlat

,zipcode,latitude,longitude
0,52075,42.173534,-90.478335
1,50483,43.103817,-93.981484
2,50207,41.446140,-92.643658
3,52163,43.217452,-92.092078
4,52202,42.146608,-91.631975
...,...,...,...
2644,46777,40.890330,-85.155009
2645,46791,40.837135,-85.268112
2646,56054,44.438841,-94.379477
2647,46795,41.548890,-85.360951


In [24]:
start = "2015-01-01"
end = "2019-12-31"

In [25]:
def get_solar_radiation_for_row(row):
    try:
        lat = row['latitude']
        lon = row['longitude']
        zip_code = row['zipcode']
    except Exception as e:
        print(f"Missing data for ZIP row: {e}")
        return []

    point = ee.Geometry.Point([lon, lat])

    solar = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET") \
        .filterDate(start, end) \
        .filterBounds(point) \
        .select("rmax", "rmin", "pr", "vs", "sph", "srad", "th", "tmmn", "tmmx", "erc", "eto", "bi", "fm100", "fm1000", "etr", "vpd")

    def extract_solar(img):
        value = img.reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=point,
            scale=4000
        )
        return ee.Feature(None, {
            'date': img.date().format('YYYY-MM-dd'),
            'HumidityMax': value.get('rmax'),
            'HumidityMin': value.get('rmin'),
            'Precipition': value.get('pr'),
            'WindSpeed': value.get('vs'),
            'Specific Humidity': value.get('sph'),
            'Radiation': value.get('srad'),
            'Wind Direction': value.get('th'),
            'Minimum temperature': value.get('tmmn'),
            'Maximum temperature': value.get('tmmx'),
            'Energy release': value.get('erc'),
            'Evapotranspiration': value.get('eto'),
            'Burning Index': value.get('bi'),
            '100-hour dead fuel moisture': value.get('fm100'),
            '1000-hour dead fuel moisture': value.get('fm1000'),
            'Daily alfalfa reference evapotranspiration': value.get('etr'),
            'Mean vapor pressure deficit': value.get('vpd')
        })

    results = solar.map(extract_solar).getInfo()

    records = []
    for f in results['features']:
        props = f['properties']
        records.append({
            'zipcode': zip_code,
            'date': props['date'],
            'HumidityMin': props['HumidityMin'],
            'HumidityMax': props['HumidityMax'],
            'Precipition': props['Precipition'],
            'WindSpeed': props['WindSpeed'],
            'Specific Humidity': props['Specific Humidity'],
            'Radiation': props['Radiation'],
            'Wind Direction': props['Wind Direction'],
            'Minimum temperature': props['Minimum temperature'],
            'Maximum temperature': props['Maximum temperature'],
            'Energy release': props['Energy release'],
            'Evapotranspiration': props['Evapotranspiration'],
            'Burning Index': props['Burning Index'],
            '100-hour dead fuel moisture': props['100-hour dead fuel moisture'],
            '1000-hour dead fuel moisture': props['1000-hour dead fuel moisture'],
            'Daily alfalfa reference evapotranspiration': props['Daily alfalfa reference evapotranspiration'],
            'Mean vapor pressure deficit': props['Mean vapor pressure deficit']
        })
    return records

In [ ]:
tqdm.pandas()

all_data = []

for _, row in tqdm(longlat.iterrows(), total=len(longlat), desc="Processing ZIPs"):
    try:
        data = get_solar_radiation_for_row(row)
        all_data.extend(data)
    except Exception as e:
        print(f"Failed for ZIP {row['zipcode']}: {e}")

df_solar = pd.DataFrame(all_data)
df_solar['date'] = pd.to_datetime(df_solar['date'])

print(df_solar.head())

Processing ZIPs:   0%|          | 0/2647 [00:00<?, ?it/s]

In [ ]:
def zip_to_latlon(zip_code):
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode({'postalcode': zip_code, 'country': 'USA'})
    if not location:
        raise ValueError(f"ZIP code {zip_code} not found")
    return location.longitude, location.latitude

# Function to extract solar radiation for a single point and date range
def get_solar_radiation_for_zip(zip_code):
    try:
        lon, lat = zip_to_latlon(zip_code)
    except Exception as e:
        print(e)
        return []  # skip this ZIP if no location found

    point = ee.Geometry.Point([lon, lat])

    # Use GRIDMET daily dataset
    solar = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET") \
        .filterDate(start, end) \
        .filterBounds(point) \
        .select("rmax", "rmin", "pr", "vs", "sph", "srad", "th", "tmmn", "tmmx", "erc", "eto", "bi", "fm100", "fm1000", "etr", "vpd")

    def extract_solar(img):
        value = img.reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=point,
            scale=4000  # GRIDMET is ~4km resolution
        )
        return ee.Feature(None, {
            'date': img.date().format('YYYY-MM-dd'),
            'HumidityMax': value.get('rmax'),
            'HumidityMin': value.get('rmin'),
            'Precipition': value.get('pr'),
            'WindSpeed': value.get('vs'),
            'Specific Humidity': value.get('sph'),
            'Radiation': value.get('srad'),
            'Wind Direction': value.get('th'),
            'Minimum temperature': value.get('tmmn'),
            'Maximum temperature': value.get('tmmx'),
            'Energy release': value.get('erc'),
            'Evapotranspiration': value.get('eto'),
            'Burning Index': value.get('bi'),
            '100-hour dead fuel moisture': value.get('fm100'),
            '1000-hour dead fuel moisture': value.get('fm1000'),
            'Daily alfalfa reference evapotranspiration': value.get('etr'),
            'Mean vapor pressure deficit': value.get('vpd')
        })

    results = solar.map(extract_solar).getInfo()

    records = []
    for f in results['features']:
        props = f['properties']
        records.append({
            'zipcode': zip_code,
            'date': props['date'],
            'HumidityMin': props['HumidityMin'],
            'HumidityMax': props['HumidityMax'],
            'Precipition': props['Precipition'],
            'WindSpeed': props['WindSpeed'],
            'Specific Humidity': props['Specific Humidity'],
            'Radiation': props['Radiation'],
            'Wind Direction': props['Wind Direction'],
            'Minimum temperature': props['Minimum temperature'],
            'Maximum temperature': props['Maximum temperature'],
            'Energy release': props['Energy release'],
            'Evapotranspiration': props['Evapotranspiration'],
            'Burning Index': props['Burning Index'],
            '100-hour dead fuel moisture': props['100-hour dead fuel moisture'],
            '1000-hour dead fuel moisture': props['1000-hour dead fuel moisture'],
            'Daily alfalfa reference evapotranspiration': props['Daily alfalfa reference evapotranspiration'],
            'Mean vapor pressure deficit': props['Mean vapor pressure deficit']
        })
    return records

# Collect data for all ZIP codes
all_data = []
for zc in zip_codes:
    print(f"Processing ZIP code {zc}")
    try:
        data_for_zip = get_solar_radiation_for_zip(zc)
        all_data.extend(data_for_zip)
    except Exception as e:
        print(f"Failed for ZIP {zc}: {e}")
    time.sleep(1)  # be kind to Nominatim API

# Create DataFrame
df_solar = pd.DataFrame(all_data)

# Convert date column to datetime
df_solar['date'] = pd.to_datetime(df_solar['date'])

print(df_solar.head())

In [ ]:
df_solar.to_csv("CornDataMoscow.csv")

In [33]:
data = pd.read_csv('CornDataMoscow.csv')
data = data.iloc[:, 1:]
data

,zipcode,date,HumidityMin,HumidityMax,Precipition,WindSpeed,Specific Humidity,Radiation,Wind Direction,Minimum temperature,Maximum temperature,Energy release,Evapotranspiration,Burning Index,100-hour dead fuel moisture,1000-hour dead fuel moisture,Daily alfalfa reference evapotranspiration,Mean vapor pressure deficit
0,51031,2019-01-01,61.500000,91.900002,0.0,4.4,0.00090,76.199997,292,254.399994,261.799988,12,0.3,19,21.600000,22.900000,0.5,0.06
1,51031,2019-01-02,48.500000,100.000000,0.0,4.3,0.00172,83.800003,221,260.100006,273.500000,14,0.7,20,21.500000,22.600000,1.2,0.16
2,51031,2019-01-03,36.700001,75.599998,0.0,3.5,0.00222,89.099998,208,268.200012,281.299988,18,1.4,21,18.900000,22.100000,2.3,0.41
3,51031,2019-01-04,42.000000,86.300003,0.0,2.5,0.00265,88.099998,268,268.799988,281.799988,19,1.0,19,18.299999,21.799999,1.7,0.37
4,51031,2019-01-05,40.400002,83.400002,0.0,2.9,0.00295,89.900002,286,270.700012,284.000000,19,1.3,20,18.000000,21.600000,2.1,0.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415955,45897,2019-12-26,43.700001,85.099998,0.0,4.5,0.00622,80.500000,151,281.600006,291.200012,27,2.0,28,15.800000,18.500000,3.2,0.60
1415956,45897,2019-12-27,55.400002,100.000000,0.0,3.0,0.00562,83.599998,268,275.500000,286.399994,25,0.8,22,17.299999,18.600000,1.3,0.23
1415957,45897,2019-12-28,47.700001,90.900002,8.2,3.0,0.00437,80.500000,78,275.500000,284.600006,14,1.1,0,19.700001,19.400000,1.7,0.35
1415958,45897,2019-12-29,62.000000,100.000000,17.6,4.5,0.00731,37.200001,93,281.399994,288.899994,9,1.2,0,22.600000,20.700001,1.8,0.28


In [34]:
soil = pd.read_csv('soil.csv')
soil.drop(columns=['Unnamed: 0'], inplace=True)
soil

,zipcode,date,soil_moisture
0,51031,2019-01-02,24.563450
1,51031,2019-01-05,23.014755
2,51031,2019-01-08,23.720436
3,51031,2019-01-11,22.587374
4,51031,2019-01-14,22.081245
...,...,...,...
470080,45897,2019-12-16,24.936007
470081,45897,2019-12-19,24.177746
470082,45897,2019-12-22,23.036732
470083,45897,2019-12-25,21.525829


In [35]:
data = data.merge(soil, on=['zipcode', 'date'], how='left')

In [36]:
data['soil_moisture'] = data['soil_moisture'].interpolate(method='linear', limit_direction='forward')
data

,zipcode,date,HumidityMin,HumidityMax,Precipition,WindSpeed,Specific Humidity,Radiation,Wind Direction,Minimum temperature,Maximum temperature,Energy release,Evapotranspiration,Burning Index,100-hour dead fuel moisture,1000-hour dead fuel moisture,Daily alfalfa reference evapotranspiration,Mean vapor pressure deficit,soil_moisture
0,51031,2019-01-01,61.500000,91.900002,0.0,4.4,0.00090,76.199997,292,254.399994,261.799988,12,0.3,19,21.600000,22.900000,0.5,0.06,NaN
1,51031,2019-01-02,48.500000,100.000000,0.0,4.3,0.00172,83.800003,221,260.100006,273.500000,14,0.7,20,21.500000,22.600000,1.2,0.16,24.563450
2,51031,2019-01-03,36.700001,75.599998,0.0,3.5,0.00222,89.099998,208,268.200012,281.299988,18,1.4,21,18.900000,22.100000,2.3,0.41,24.047218
3,51031,2019-01-04,42.000000,86.300003,0.0,2.5,0.00265,88.099998,268,268.799988,281.799988,19,1.0,19,18.299999,21.799999,1.7,0.37,23.530987
4,51031,2019-01-05,40.400002,83.400002,0.0,2.9,0.00295,89.900002,286,270.700012,284.000000,19,1.3,20,18.000000,21.600000,2.1,0.44,23.014755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415955,45897,2019-12-26,43.700001,85.099998,0.0,4.5,0.00622,80.500000,151,281.600006,291.200012,27,2.0,28,15.800000,18.500000,3.2,0.60,22.309567
1415956,45897,2019-12-27,55.400002,100.000000,0.0,3.0,0.00562,83.599998,268,275.500000,286.399994,25,0.8,22,17.299999,18.600000,1.3,0.23,23.093305
1415957,45897,2019-12-28,47.700001,90.900002,8.2,3.0,0.00437,80.500000,78,275.500000,284.600006,14,1.1,0,19.700001,19.400000,1.7,0.35,23.877043
1415958,45897,2019-12-29,62.000000,100.000000,17.6,4.5,0.00731,37.200001,93,281.399994,288.899994,9,1.2,0,22.600000,20.700001,1.8,0.28,23.877043


In [37]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd

# Replace with your actual API key
api_key = 'KDDFZLG0L20J7ABF'

# Create TimeSeries object
ts = TimeSeries(key=api_key, output_format='pandas')

# Get daily data for CORN ETF
price, meta = ts.get_daily(symbol='CORN', outputsize='full')

# Rename columns for clarity
price.columns = ['open', 'high', 'low', 'close', 'volume']

# Sort by date first
price = price.sort_index()

# Then filter for 2019 only
price = price.loc['2019-01-01':'2019-12-31']

print(price.head())

             open     high    low  close    volume
date                                              
2019-01-02  16.10  16.1700  16.03  16.12    9708.0
2019-01-03  16.14  16.2799  16.14  16.25  135468.0
2019-01-04  16.28  16.4000  16.28  16.40   90657.0
2019-01-07  16.42  16.4200  16.35  16.40   41094.0
2019-01-08  16.43  16.4399  16.28  16.33  147125.0


In [38]:
price['MeanPrice'] = (price['open'] + price['close']) / 2

In [43]:
# Reset index in price to get 'Date' as a column
price = price.reset_index()

# Extract relevant columns and rename 'Date' to 'date'
mean_price = price[['date', 'MeanPrice']].copy()
mean_price.rename(columns={'Date': 'date'}, inplace=True)

# Convert both date columns to datetime (safe for merging)
data['date'] = pd.to_datetime(data['date'])
mean_price['date'] = pd.to_datetime(mean_price['date'])

# Merge mean_price into data on 'date'
data = pd.merge(data, mean_price, on='date', how='left')


In [44]:
data

,zipcode,date,HumidityMin,HumidityMax,Precipition,WindSpeed,Specific Humidity,Radiation,Wind Direction,Minimum temperature,Maximum temperature,Energy release,Evapotranspiration,Burning Index,100-hour dead fuel moisture,1000-hour dead fuel moisture,Daily alfalfa reference evapotranspiration,Mean vapor pressure deficit,soil_moisture,MeanPrice
0,51031,2019-01-01,61.500000,91.900002,0.0,4.4,0.00090,76.199997,292,254.399994,261.799988,12,0.3,19,21.600000,22.900000,0.5,0.06,NaN,NaN
1,51031,2019-01-02,48.500000,100.000000,0.0,4.3,0.00172,83.800003,221,260.100006,273.500000,14,0.7,20,21.500000,22.600000,1.2,0.16,24.563450,16.110
2,51031,2019-01-03,36.700001,75.599998,0.0,3.5,0.00222,89.099998,208,268.200012,281.299988,18,1.4,21,18.900000,22.100000,2.3,0.41,24.047218,16.195
3,51031,2019-01-04,42.000000,86.300003,0.0,2.5,0.00265,88.099998,268,268.799988,281.799988,19,1.0,19,18.299999,21.799999,1.7,0.37,23.530987,16.340
4,51031,2019-01-05,40.400002,83.400002,0.0,2.9,0.00295,89.900002,286,270.700012,284.000000,19,1.3,20,18.000000,21.600000,2.1,0.44,23.014755,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415955,45897,2019-12-26,43.700001,85.099998,0.0,4.5,0.00622,80.500000,151,281.600006,291.200012,27,2.0,28,15.800000,18.500000,3.2,0.60,22.309567,14.820
1415956,45897,2019-12-27,55.400002,100.000000,0.0,3.0,0.00562,83.599998,268,275.500000,286.399994,25,0.8,22,17.299999,18.600000,1.3,0.23,23.093305,14.865
1415957,45897,2019-12-28,47.700001,90.900002,8.2,3.0,0.00437,80.500000,78,275.500000,284.600006,14,1.1,0,19.700001,19.400000,1.7,0.35,23.877043,NaN
1415958,45897,2019-12-29,62.000000,100.000000,17.6,4.5,0.00731,37.200001,93,281.399994,288.899994,9,1.2,0,22.600000,20.700001,1.8,0.28,23.877043,NaN


In [45]:
data.to_csv('MLprojectData.csv')